In [2]:
import numpy as np

# class optStruct:
    def __init__(self,dataMatIn,classLabels,C,toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m,1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m,2)))
        self.K = np.mat(np.zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X,self.X[i,:],kTup)
        

In [73]:
def calcEk(oS,k):

    fk = float(np.multiply(oS.alphas,oS.labelMat).T*oS.K[:,k]+oS.b)
    Ek = fk - float(oS.labelMat[k]) 
    return Ek

In [62]:
def selectJrand(i,m):#随机选择一个和i不同的j
    j=i
    while(j==i):
        j=int(np.random.uniform(0,m))
    return j

In [63]:
def selectJ(i,oS,Ei):
    maxk = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1,Ei]
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]
    if len(validEcacheList) > 1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS,k)
            deltaE = abs(Ei-Ek)
            if( deltaE>maxDeltaE):
                maxDeltaE = deltaE
                maxk = k
                Ej = Ek
        return maxk,Ej
    else:
        j=selectJrand(i,oS.m)#随机选择一个和i不同的j
        Ej=calcEk(oS,j)#计算误差
    return j,Ej

In [64]:
def updateEk(oS,k):
    Ek = calcEk(oS,k)
    oS.eCache[k] = [1,Ek]

In [65]:
def clipAlpha(alpha,H,L):
    if alpha < L:
        return L
    elif alpha > H:
        return H
    else :
        return alpha

In [79]:
def innerL(i,oS):
    Ei = calcEk(oS,i)
    if((oS.labelMat[i]*Ei<-oS.tol) and (oS.alphas[i]<oS.C)) or ((oS.labelMat[i]*Ei>oS.tol)
                                                               and (oS.alphas[i]>0)):
        j,Ej = selectJ(i,oS,Ei)
        alphaOldI = oS.alphas[i].copy()
        alphaOldJ = oS.alphas[j].copy()
        if(oS.labelMat[i]!=oS.labelMat[j]):
            L = max(0,oS.alphas[j] - oS.alphas[i])
            H = min(oS.C,oS.C-oS.alphas[i]+oS.alphas[j])
        else:
            L = max(0,oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C,oS.alphas[j] + oS.alphas[i])
            
        if L==H :
            print("L==H")
            return 0
        eta = 2.0*oS.K[i,j]-oS.K[i,i]-oS.K[j,j]
        oS.alphas[j] -= oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        
        if(abs(oS.alphas[j]-alphaOldJ)<0.00001):
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaOldJ-oS.alphas[j])
        updateEk(oS,i)
        
        b1 = oS.b - Ei - oS.labelMat[i]*oS.K[i,i]*(oS.alphas[i]-alphaOldI)-oS.labelMat[j]*oS.K[i,j]*(oS.alphas[j]-alphaOldJ)
        

        b2 = oS.b - Ej-oS.labelMat[i]*oS.K[i,j]*(oS.alphas[i]-alphaOldI)-oS.labelMat[j]*oS.K[j,j]*(oS.alphas[j]-alphaOldJ)
        
        
        if(0<oS.alphas[i]) and (oS.alphas[i]<oS.C):
            oS.b = b1
        elif (0<oS.alphas[j]) and (oS.alphas[j]<oS.C):
            oS.b = b2
        else:
            oS.b = (b1+b1)/2
        
        return 1
    else:
        return 0

In [80]:
def smoP(dataMatIn,classLabels,C,toler,MaxIter,kTup=('lin',0)):
    oS = optStruct(np.mat(dataMatIn),np.mat(classLabels).transpose(),C,toler)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    
    while(iter<MaxIter) and ((alphaPairsChanged>0) or (entireSet)):
        alphaPairsChanged = 0 
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
                print("fullSet,iter:%d i:%d,pairs changed %d"%(iter,i,alphaPairsChanged))
            iter +=1
        else:
            nonBoundIs = np.nonzero((oS.alphas.A>0) * (oS.alphas.A<C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("non-bound,iter:%d i:%d,pairs changed %d"%(iter,i,alphaPairsChanged))
            iter += 1
                
        if entireSet:
            entireSet = False
        elif alphaPairsChanged==0:
            entireSet = True
            
        print("iteration number:%d"%iter)
    return oS.b,oS.alphas

In [68]:
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

In [69]:
def calcWs(alphas,dataArr,classLabel):
    X = np.mat(dataArr)
    labelMat = np.mat(classLabel).transpose()
    m,n = np.shape(X)
    w = np.zeros((n,1))
    for i in range(m):
        w += np.multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [81]:
dataArr,labelArr = loadDataSet("testSet.txt")

# labelArr = np.mat(labelArr)



In [82]:
b,alphas = smoP(dataArr,labelArr,0.6,0.001,40)
ws = calcWs(alphas,dataArr,labelArr)
print(ws)


L==H
fullSet,iter:0 i:0,pairs changed 0
L==H
fullSet,iter:0 i:1,pairs changed 0
fullSet,iter:0 i:2,pairs changed 1
L==H
fullSet,iter:0 i:3,pairs changed 1
fullSet,iter:0 i:4,pairs changed 2
fullSet,iter:0 i:5,pairs changed 2
fullSet,iter:0 i:6,pairs changed 2
j not moving enough
fullSet,iter:0 i:7,pairs changed 2
L==H
fullSet,iter:0 i:8,pairs changed 2
fullSet,iter:0 i:9,pairs changed 2
L==H
fullSet,iter:0 i:10,pairs changed 2
L==H
fullSet,iter:0 i:11,pairs changed 2
L==H
fullSet,iter:0 i:12,pairs changed 2
fullSet,iter:0 i:13,pairs changed 2
L==H
fullSet,iter:0 i:14,pairs changed 2
fullSet,iter:0 i:15,pairs changed 2
fullSet,iter:0 i:16,pairs changed 2
L==H
fullSet,iter:0 i:17,pairs changed 2
fullSet,iter:0 i:18,pairs changed 3
fullSet,iter:0 i:19,pairs changed 3
fullSet,iter:0 i:20,pairs changed 3
fullSet,iter:0 i:21,pairs changed 3
j not moving enough
fullSet,iter:0 i:22,pairs changed 3
L==H
fullSet,iter:0 i:23,pairs changed 3
L==H
fullSet,iter:0 i:24,pairs changed 3
fullSet,iter:0 

In [83]:
dataMat = np.mat(dataArr)
print(dataMat[0]*np.mat(ws)+b)
print(labelArr[0])

[[-0.92555695]]
-1.0


In [1]:
def kernelTrans(X,A,kTup):
    m,n = np.shape(X)
    K = np.mat(np.zeors(m,1))
    if kTup[0] == 'lin':
        K = X*A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = exp(K/(-1*kTup[1]**2))
    else:
        raise NameError('Houston We Have a Problem That Kernal is not recognized')
    return K

# print(dataMat[0,:].T n'n'm